# Sleeping Position and Sudden Infant Death Syndrome

**Date:** 2021-11-28

**Reference:** M249, Book 1, Part 1

In [1]:
suppressPackageStartupMessages(library(tidyverse))
library(R249)

## Summary

The aim of this notebook was to show how to analyse a case-control study with multiple exposures in **R**.
Data on a case-control study examining cot death and the last placed sleeping position of an infant were gathered.
The odds ratio was calculated and a chi-squared test of no association was performed.
It was concluded that the study provided strong evidence of a positive association between cot death and placing an infants down on it front or side, and a negative associaition between cot death and placing an infants down on it front or back.

## Get the data

Researchers gather data on 188 cases of infants who died of SIDS and 774 controls of live infants of similar ages and from the same localities as the babies that died.
The exposures were the last placed sleeping position of the infant, one of either side, front, back.
The reference category for this analysis was chosen to be infants who slept on their side.

Results from the cohort study are saved in the `data` folder.

In [2]:
(res <- as_tibble(read.csv(file = "..\\..\\data\\sids.csv")))

count,exposure,outcome
<int>,<chr>,<chr>
30,slept on front,case
24,slept on front,control
76,slept on side,case
241,slept on side,control
82,slept on back,case
509,slept on back,control


## Prepare the data

Cast the `exposure`, `casecon` columns to factors.

In [3]:
labexp <- c("slept on side", "slept on front", "slept on back")
labout <- c("control", "case")
(sortedres <- res %>%
    mutate(exposure = factor(res$exposure, labexp)) %>%
    mutate(outcome = factor(res$outcome, labout)) %>%
    arrange(exposure, outcome))

count,exposure,outcome
<int>,<fct>,<fct>
241,slept on side,control
76,slept on side,case
24,slept on front,control
30,slept on front,case
509,slept on back,control
82,slept on back,case


Pull the `count` column as a vector and use it to initialise a matrix.

In [4]:
(resmat <- sortedres %>%
    pull(count) %>%
    matrix(nrow = 3, ncol = 2, byrow = TRUE, dimnames = list(labexp, labout)))

,control,case
slept on side,241,76
slept on front,24,30
slept on back,509,82


## Measures of assocation

Calculate the **odds ratio**.
Note that the _relative risk_ is not defined for a case-control study.

In [5]:
oddsratio(resmat, alpha = 0.01)

,oddsratio,stderr,lcb,ucb
exposure 1 (-),NA,NA,NA,NA
exposure 2 (+),3.9638158,0.3038211,1.812335,8.6693905
exposure 3 (+),0.5108572,0.1773895,0.323489,0.8067511


This suggests there is a positive association between babies placed to sleep on their fronts and cot deaths: the odds ratio is **3.96**, with 99% confidence interval **(1.81, 8.67)** located well above **1**.
Conversely, there is a negative association between babies placed to sleep on their backs and cot deaths: the odds ratio is **0.51**, with 99% confidence interval **(0.18, 0.32)** located well below **1**.

## Strength of evidence of association

Perform a chi-squared test of no association.

In [6]:
chi2 <- chisq.test(resmat, correct = FALSE)

Calculate the expected frequencies under a null hypothesis of no association.

In [7]:
chi2$expected

,control,case
slept on side,255.04990,61.95010
slept on front,43.44699,10.55301
slept on back,475.50312,115.49688


The numbers of cases observed are greater than expected for babies placed on their front or side, but much less than expected for those babies placed on the back.
This suggests that there might be a positive association between SIDS and placing a baby down on its front or side, and a negative associaition between SIDS and placing a baby down on its back.

The null distribution of the test statistic is approximately **chi-squared** with **2** degrees of freedom.
Since all expected frequencies are greater than **5**, the approximation is adequate.

Contribution of the values to the **chi-squared** test statistic.

In [8]:
(chi2$residuals) ** 2

,control,case
slept on side,0.7739646,3.186429
slept on front,8.7045221,35.836702
slept on back,2.3596923,9.714904


Return the test statistic and **p**-value.

In [9]:
chi2


	Pearson's Chi-squared test

data:  resmat
X-squared = 60.576, df = 2, p-value = 7.015e-14


The test statistic is approximately **60.58**, with **p**-value less than **0.000000**.
There is very strong evidence against the null hypothesis of no association.
Hence, there is strong evidence of an associaition between the last placed sleeping position of an infant and cot death.
There is evidence of a positive association for babies left to sleep on their front or side, and a negative associaition for those left to sleep on their back.